In [1]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import confusion_matrix
from keras.utils.vis_utils import plot_model

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from keras.utils import np_utils


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

import seaborn as sns
import matplotlib.pyplot as plt
import os

2022-05-09 09:12:50.458403: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/jest/anaconda3/envs/javeriana_tf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir(r"/home/jest/Documents/personal/sistemas inteligentes/proyecto/segunda entrega")

In [3]:
path_name_model = 'dataset/datos_para_modelo.csv'

In [4]:
df = pd.read_csv(path_name_model, sep='|')

In [5]:
df = df.iloc[:,1:]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4526 entries, 0 to 4525
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   estu_genero                   4526 non-null   object 
 1   estu_tieneetnia               4526 non-null   object 
 2   estu_depto_reside             4526 non-null   object 
 3   estu_mcpio_reside             4526 non-null   object 
 4   fami_estratovivienda          4526 non-null   object 
 5   fami_personashogar            4526 non-null   object 
 6   fami_cuartoshogar             4526 non-null   object 
 7   fami_tieneinternet            4526 non-null   object 
 8   fami_tieneserviciotv          4526 non-null   object 
 9   fami_tienecomputador          4526 non-null   object 
 10  fami_tienelavadora            4526 non-null   object 
 11  fami_tienehornomicroogas      4526 non-null   object 
 12  fami_tieneautomovil           4526 non-null   object 
 13  fam

In [7]:
dataset = df.values

In [8]:
X = dataset[:, :-1]
y = dataset[:,-1]
X = X.astype(str)
y = y.reshape((len(y), 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (3168, 22) (3168, 1)
Test (1358, 22) (1358, 1)


In [9]:
X_test

array([['F', 'No', 'BOLIVAR', ..., '40.0', '37.0', '41.0'],
       ['M', 'No', 'BOLIVAR', ..., '32.0', '26.0', '31.0'],
       ['F', 'No', 'BOLIVAR', ..., '45.0', '41.0', '47.0'],
       ...,
       ['F', 'No', 'BOLIVAR', ..., '45.0', '35.0', '36.0'],
       ['M', 'No', 'BOLIVAR', ..., '30.0', '40.0', '52.0'],
       ['M', 'No', 'BOLIVAR', ..., '32.0', '41.0', '52.0']], dtype='<U21')

In [10]:
oe = OrdinalEncoder()
oe.fit(X_train)
X_train_enc = oe.transform(X_train)
oe.fit(X_test)
X_test_enc = oe.transform(X_test)

In [11]:
le = LabelEncoder()
le.fit(y_train)
y_train_enc = le.transform(y_train)
y_test_enc = le.transform(y_test)
y_train_enc = np_utils.to_categorical(y_train_enc)
y_test_enc = np_utils.to_categorical(y_test_enc)

/home/jest/anaconda3/envs/javeriana_tf/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jest/anaconda3/envs/javeriana_tf/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Modelo bayesiano 

In [34]:

import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
import matplotlib.pyplot as plt
%matplotlib inline

from collections import OrderedDict



In [48]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_enc = le.fit_transform(y_train)
# targets: array([0, 1, 2, 3])
y_train_enc
#targets = torch.as_tensor(targets)

array([2, 0, 0, ..., 1, 1, 1])

In [56]:
X = X_train
Y = y_train_enc
x = torch.from_numpy(X_train_enc).float()
y = torch.from_numpy(y_train_enc).long()
print(x.shape, y.shape)

torch.Size([3168, 22]) torch.Size([3168])


# Test Modelo Bayes

In [66]:
model = nn.Sequential(nn.Sequential(OrderedDict([
    ('capa_bayes_1',bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=22, out_features=64)),# Num features
    ('sigmoide_1', nn.Sigmoid()),       
    ('capa_bayes_2',bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=64, out_features=16)),# Num features
    ('sigmoide_2', nn.Sigmoid()),
    ('capa_bayes_3',bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=16, out_features=3)),# Num features
    ('sigmoide_3', nn.Sigmoid()),
        ])))


In [67]:
print(model)

Sequential(
  (0): Sequential(
    (capa_bayes_1): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=22, out_features=64, bias=True)
    (sigmoide_1): Sigmoid()
    (capa_bayes_2): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=64, out_features=16, bias=True)
    (sigmoide_2): Sigmoid()
    (capa_bayes_3): BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=16, out_features=3, bias=True)
    (sigmoide_3): Sigmoid()
  )
)


In [68]:
ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [69]:
%%time
kl_weight = 0.1
for step in range(3000):
    pre = model(x)
    ce = ce_loss(pre, y)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
_, predicted = torch.max(pre.data, 1)
total = y.size(0)
correct = (predicted == y).sum()
print('- Accuracy: %f %%' % (100 * float(correct) / total))
print('- CE : %2.2f, KL : %2.2f' % (ce.item(), kl.item()))


- Accuracy: 80.934343 %
- CE : 0.73, KL : 1.05
CPU times: user 1min 15s, sys: 755 ms, total: 1min 16s
Wall time: 19.2 s


# Experimentos Bayes

In [72]:
def red_bayesiana(reduction_ = 'mean', kl_weight = 0.01, lr_= 0.01, prior_mu_ = 0, prior_sigma_ = 0.1, step_ = 3000):
    model = nn.Sequential(nn.Sequential(OrderedDict([
    ('capa_bayes_1',bnn.BayesLinear(prior_mu=prior_mu_, prior_sigma=prior_sigma_, in_features=22, out_features=64)),# Num features
    ('sigmoide_1', nn.Sigmoid()),       
    ('capa_bayes_2',bnn.BayesLinear(prior_mu=prior_mu_, prior_sigma=prior_sigma_, in_features=64, out_features=16)),# Num features
    ('sigmoide_2', nn.Sigmoid()),
    ('capa_bayes_3',bnn.BayesLinear(prior_mu=prior_mu_, prior_sigma=prior_sigma_, in_features=16, out_features=3)),# Num features
    ('sigmoide_3', nn.Sigmoid()),
        ])))
    
    ce_loss = nn.CrossEntropyLoss()
    kl_loss = bnn.BKLLoss(reduction=reduction_, last_layer_only=False)
    
    optimizer = optim.Adam(model.parameters(), lr=lr_)
    

    for step in range(step_):
        pre = model(x)
        ce = ce_loss(pre, y)
        kl = kl_loss(model)
        cost = ce + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    _, predicted = torch.max(pre.data, 1)
    total = y.size(0)
    correct = (predicted == y).sum()
    print('- Accuracy: %f %%' % (100 * float(correct) / total))
    print('- CE : %2.2f, KL : %2.2f' % (ce.item(), kl.item()))
    print('-'*100)



In [73]:
red_bayesiana(reduction_ = 'mean', kl_weight = 0.01, lr_= 0.01, prior_mu_ = 0, prior_sigma_ = 0.1, step_ = 3000)

- Accuracy: 81.786616 %
- CE : 0.65, KL : 3.61
----------------------------------------------------------------------------------------------------


In [74]:
red_bayesiana(reduction_ = 'sum', kl_weight = 0.01, lr_= 0.01, prior_mu_ = 0, prior_sigma_ = 0.1, step_ = 3000)

- Accuracy: 54.987374 %
- CE : 1.09, KL : 1.40
----------------------------------------------------------------------------------------------------


In [75]:
red_bayesiana(reduction_ = 'mean', kl_weight = 0.01, lr_= 0.01, prior_mu_ = 0, prior_sigma_ = 0.01, step_ = 3000)

- Accuracy: 54.987374 %
- CE : 1.00, KL : 2.54
----------------------------------------------------------------------------------------------------


In [76]:
red_bayesiana(reduction_ = 'mean', kl_weight = 0.01, lr_= 0.01, prior_mu_ = 0, prior_sigma_ = 0.5, step_ = 3000)

- Accuracy: 81.755051 %
- CE : 0.62, KL : 0.83
----------------------------------------------------------------------------------------------------


In [78]:
red_bayesiana(reduction_ = 'mean', kl_weight = 0.01, lr_= 0.01, prior_mu_ = -0.5, prior_sigma_ = 1, step_ = 3000)

- Accuracy: 72.190657 %
- CE : 0.76, KL : 0.56
----------------------------------------------------------------------------------------------------


In [80]:
red_bayesiana(reduction_ = 'mean', kl_weight = 0.01, lr_= 1, prior_mu_ = 0, prior_sigma_ = 1, step_ = 3000)

- Accuracy: 54.987374 %
- CE : 1.00, KL : 2.86
----------------------------------------------------------------------------------------------------
